In [1]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape,RepeatVector,TimeDistributed
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout,GlobalMaxPooling1D

from utils.data_split import split_sequence_parallel
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 不加这几句，则CONV 报错
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Using TensorFlow backend.


In [2]:
data = pd.read_csv("D:\桌面\空气质量预测综合\北京多站点空气质量数据集\北京天坛\\InfoGainTiantandata.csv", header=0,infer_datetime_format=True, engine='python')
data['Unnamed: 0']=pd.to_datetime(data['Unnamed: 0'])
data.set_index("Unnamed: 0",inplace=True)

In [3]:
'''
归一化
'''
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler(feature_range=(0, 1))
scaler2 = MinMaxScaler(feature_range=(0, 1))
scaler3 = MinMaxScaler(feature_range=(0, 1))
scaler4 = MinMaxScaler(feature_range=(0, 1))
scaler5 = MinMaxScaler(feature_range=(0, 1))
scaler6 = MinMaxScaler(feature_range=(0, 1))
scaler7 = MinMaxScaler(feature_range=(0, 1))
scaler8 = MinMaxScaler(feature_range=(0, 1))
scaler9 = MinMaxScaler(feature_range=(0, 1))
scaler10 = MinMaxScaler(feature_range=(0, 1))
scaler11 = MinMaxScaler(feature_range=(0, 1))
scaler12 = MinMaxScaler(feature_range=(0, 1))
scaler13 = MinMaxScaler(feature_range=(0, 1))
scaler14 = MinMaxScaler(feature_range=(0, 1))
scaler15 = MinMaxScaler(feature_range=(0, 1))
data_minmax = data.copy()
data_minmax['PM2.5']=scaler1.fit_transform(data_minmax['PM2.5'].values.reshape(-1,1))
data_minmax['PM10']=scaler2.fit_transform(data_minmax['PM10'].values.reshape(-1,1))
data_minmax['SO2']=scaler3.fit_transform(data_minmax['SO2'].values.reshape(-1,1))
data_minmax['NO2']=scaler4.fit_transform(data_minmax['NO2'].values.reshape(-1,1))
data_minmax['CO']=scaler5.fit_transform(data_minmax['CO'].values.reshape(-1,1))
data_minmax['O3']=scaler6.fit_transform(data_minmax['O3'].values.reshape(-1,1))
data_minmax['TEMP']=scaler7.fit_transform(data_minmax['TEMP'].values.reshape(-1,1))
data_minmax['PRES']=scaler8.fit_transform(data_minmax['PRES'].values.reshape(-1,1))
data_minmax['DEWP']=scaler9.fit_transform(data_minmax['DEWP'].values.reshape(-1,1))
data_minmax['wd_E']=scaler10.fit_transform(data_minmax['wd_E'].values.reshape(-1,1))
data_minmax['wd_ENE']=scaler11.fit_transform(data_minmax['wd_ENE'].values.reshape(-1,1))
data_minmax['wd_ESE']=scaler12.fit_transform(data_minmax['wd_ESE'].values.reshape(-1,1))
data_minmax['wd_NE']=scaler13.fit_transform(data_minmax['wd_NE'].values.reshape(-1,1))
data_minmax['wd_NW']=scaler14.fit_transform(data_minmax['wd_NW'].values.reshape(-1,1))
data_minmax['wd_SW']=scaler15.fit_transform(data_minmax['wd_SW'].values.reshape(-1,1))

In [4]:
'''
取一年的时间，10个月做训练和验证，2个月做测试
'''
cast1 = -10200
cast2 = -2880
cast3 = -1416                                                                      
data_train = data_minmax[cast1:cast2]
data_test = data_minmax[cast2:cast3] 
data_train.shape,data_test.shape

((7320, 15), (1464, 15))

In [5]:
data_test = np.array(data_test)
data_train = np.array(data_train)
data_train.shape,data_test.shape

((7320, 15), (1464, 15))

In [6]:
sw_width = 21
pred_length = 12
verbose_set = 2
X, y, features = split_sequence_parallel(data_train,sw_width,pred_length)
test_x,test_y,test_features = split_sequence_parallel(data_test,sw_width,pred_length)

In [7]:
def ED_LSTM_model():
#     ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))
    ip = Input(shape=(sw_width,features))

    x = LSTM(128,activation="relu")(ip)
#     x = Dropout(0.2)(x)
    x = RepeatVector(pred_length)(x)
    x = LSTM(64,activation="relu",return_sequences=True)(x)
#     x = Dropout(0.2)(x)
    
    
    out = TimeDistributed(Dense(1))(x)

    model = Model(ip, out)
    model.summary()


    return model

In [8]:
model = ED_LSTM_model()
model

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 21, 15)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               73728     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 12, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 12, 64)            49408     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 12, 1)             65        
Total params: 123,201
Trainable params: 123,201
Non-trainable params: 0
_________________________________________________________________


In [9]:
y = y.reshape(y.shape[0],y.shape[1],1)

In [10]:
import time
from keras import backend as K
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau

# lr_scheduler = LearningRateScheduler(lr_schedule)
# lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                cooldown=0,
#                                patience=5,
#                                min_lr=0.5e-6)
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

filepath = "D:\桌面\空气质量预测综合\北京多站点空气质量数据集\北京天坛\\12hourweights\encoder_decoder_weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)
callbacks_list = [checkpoint]
starttime=time.time()
model.compile(loss=root_mean_squared_error, optimizer='adam')
history1 = model.fit(X,y,validation_split=0.25,epochs=70, batch_size=32,callbacks=callbacks_list, verbose=2)
endtime=time.time()
dtime=endtime-starttime
print("程序运行时间为：%.8s s" % dtime)  #时间显示到微秒

Train on 5466 samples, validate on 1822 samples
Epoch 1/70
 - 6s - loss: 0.0586 - val_loss: 0.0400

Epoch 00001: val_loss improved from inf to 0.03999, saving model to D:\桌面\空气质量预测综合\北京多站点空气质量数据集\北京天坛\12hourweights\encoder_decoder_weights_best.hdf5
Epoch 2/70
 - 5s - loss: 0.0525 - val_loss: 0.0389

Epoch 00002: val_loss improved from 0.03999 to 0.03887, saving model to D:\桌面\空气质量预测综合\北京多站点空气质量数据集\北京天坛\12hourweights\encoder_decoder_weights_best.hdf5
Epoch 3/70
 - 5s - loss: 0.0504 - val_loss: 0.0399

Epoch 00003: val_loss did not improve from 0.03887
Epoch 4/70
 - 5s - loss: 0.0485 - val_loss: 0.0394

Epoch 00004: val_loss did not improve from 0.03887
Epoch 5/70
 - 5s - loss: 0.0463 - val_loss: 0.0429

Epoch 00005: val_loss did not improve from 0.03887
Epoch 6/70
 - 5s - loss: 0.0456 - val_loss: 0.0472

Epoch 00006: val_loss did not improve from 0.03887
Epoch 7/70
 - 5s - loss: 0.0449 - val_loss: 0.0398

Epoch 00007: val_loss did not improve from 0.03887
Epoch 8/70
 - 5s - loss: 0.0427

In [11]:
model.load_weights(filepath)

In [12]:
test_y = test_y.reshape(test_y.shape[0],test_y.shape[1],1)

In [13]:
predict=model.predict(test_x)
predict=scaler1.inverse_transform(predict.reshape(predict.shape[0],predict.shape[1]))

In [14]:
print(tf.sqrt(tf.losses.mean_squared_error(predict[:,:6],scaler1.inverse_transform(test_y.reshape(test_y.shape[0],test_y.shape[1]))[:,:6])).numpy().mean())
print(tf.losses.mae(predict[:,:6],scaler1.inverse_transform(test_y.reshape(test_y.shape[0],test_y.shape[1]))[:,:6]).numpy().mean())
print(tf.sqrt(tf.losses.mean_squared_error(predict[:,6:],scaler1.inverse_transform(test_y.reshape(test_y.shape[0],test_y.shape[1]))[:,6:])).numpy().mean())
print(tf.losses.mae(predict[:,6:],scaler1.inverse_transform(test_y.reshape(test_y.shape[0],test_y.shape[1]))[:,6:]).numpy().mean())

44.51058292038064
39.93454304363733
63.05240234043444
59.599364880618104
